In [4]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 3.6 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier


s3= boto3.resource('s3')
bucket_name= 'morgangant-bata-445-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'train.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
project = pd.read_csv(file_content_stream)
project.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemsVoids,scansWithoutRegistration,quanitityModification,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [6]:
#Dropping na values
project = project.dropna()

In [7]:
importances= list()
for i in range (1,100):
    ## Defining the input and taregt variables
    x= project[['trustLevel', 'totalScanTimeInSeconds', 'grandTotal', 'lineItemsVoids', 'scansWithoutRegistration', 'quanitityModification', 'scannedLineItemsPerSecond', 'valuePerSecond', 'lineItemVoidsPerPosition']]
    y= project['fraud']

    #Splitting data into train and test
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2)

    #Building Model
    RF_md= RandomForestClassifier(n_estimators= 500).fit(x_train, y_train)

    #Extracting the feature importances
    importances.append(RF_md.feature_importances_)

In [8]:
a = pd.DataFrame(importances)
a.columns= ['trustLevel', 'totalScanTimeInSeconds', 'grandTotal', 'lineItemsVoids', 'scansWithoutRegistration', 'quanitityModification', 'scannedLineItemsPerSecond', 'valuePerSecond', 'lineItemVoidsPerPosition']
a.apply(np.mean, axis = 0)

trustLevel                   0.267337
totalScanTimeInSeconds       0.146603
grandTotal                   0.069167
lineItemsVoids               0.070835
scansWithoutRegistration     0.054783
quanitityModification        0.024785
scannedLineItemsPerSecond    0.166051
valuePerSecond               0.091405
lineItemVoidsPerPosition     0.109034
dtype: float64

In [8]:
md1_results= list()
md2_results= list()
md3_results= list()

#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']
    

for i in range (0,1000):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Creating RF model with depth 3
    RF_md= RandomForestClassifier(max_depth= 3, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred= RF_md.predict_proba(x_test)[:,1]
    RF_pred
    #Changling likleyhoods to labels
    RF_label= np.where(RF_pred < .1, 0, 1)
    #Computing recall
    Model1_recall= recall_score(y_test, RF_label)
    md1_results.append(Model1_recall)


    #Creating RF model with depth 5
    RF_md2= RandomForestClassifier(max_depth= 5, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred2= RF_md2.predict_proba(x_test)[:,1]
    RF_pred2
    #Changling likleyhoods to labels
    RF_label2= np.where(RF_pred2 < .1, 0, 1)
    #Computing recall
    Model2_recall= recall_score(y_test, RF_label2)
    md2_results.append(Model2_recall)


    #Creating RF model with depth 5
    RF_md3= RandomForestClassifier(max_depth= 7, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred3= RF_md3.predict_proba(x_test)[:,1]
    RF_pred3
    #Changling likleyhoods to labels
    RF_label3= np.where(RF_pred3 < .1, 0, 1)
    #Computing recall
    Model3_recall= recall_score(y_test, RF_label3)
    md3_results.append(Model3_recall)

KeyboardInterrupt: 

In [4]:
#Computing averages
print('Average Recall for model 1:', np.mean(md1_results))
print('Average Recall for model 2:',np.mean(md2_results))
print('Average Recall for model 3:',np.mean(md3_results))

Average Recall for model 1: 0.8852380952380952
Average Recall for model 2: 0.9371428571428569
Average Recall for model 3: 0.9414285714285714


In [9]:
#Trying different number of trees
md1_results= list()
md2_results= list()
md3_results= list()

#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']
    
#depth of 7

for i in range (0,1000):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Creating RF model with 100 estimators
    RF_md= RandomForestClassifier(max_depth= 7, n_estimators= 100).fit(x_train, y_train)
    #Predictingon the model
    RF_pred= RF_md.predict_proba(x_test)[:,1]
    RF_pred
    #Changling likleyhoods to labels
    RF_label= np.where(RF_pred < .1, 0, 1)
    #Computing recall
    Model1_recall= recall_score(y_test, RF_label)
    md1_results.append(Model1_recall)


    #Creating RF model with 300 estimators
    RF_md2= RandomForestClassifier(max_depth= 7, n_estimators= 300).fit(x_train, y_train)
    #Predictingon the model
    RF_pred2= RF_md2.predict_proba(x_test)[:,1]
    RF_pred2
    #Changling likleyhoods to labels
    RF_label2= np.where(RF_pred2 < .1, 0, 1)
    #Computing recall
    Model2_recall= recall_score(y_test, RF_label2)
    md2_results.append(Model2_recall)


    #Creating RF model with 500 estimators
    RF_md3= RandomForestClassifier(max_depth= 7, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred3= RF_md3.predict_proba(x_test)[:,1]
    RF_pred3
    #Changling likleyhoods to labels
    RF_label3= np.where(RF_pred3 < .1, 0, 1)
    #Computing recall
    Model3_recall= recall_score(y_test, RF_label3)
    md3_results.append(Model3_recall)

In [10]:
#Computing averages
print('Average Recall for model 1:', np.mean(md1_results))
print('Average Recall for model 2:',np.mean(md2_results))
print('Average Recall for model 3:',np.mean(md3_results))

Average Recall for model 1: 0.9393333333333335
Average Recall for model 2: 0.9387619047619048
Average Recall for model 3: 0.9390952380952382


In [13]:
#Trying different number of trees
md1_results= list()
md2_results= list()
md3_results= list()

#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']
    
#depth of 3
    
for i in range (0,1000):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Creating RF model with 100 estimators
    RF_md= RandomForestClassifier(max_depth= 3, n_estimators= 100).fit(x_train, y_train)
    #Predictingon the model
    RF_pred= RF_md.predict_proba(x_test)[:,1]
    RF_pred
    #Changling likleyhoods to labels
    RF_label= np.where(RF_pred < .1, 0, 1)
    #Computing recall
    Model1_recall= recall_score(y_test, RF_label)
    md1_results.append(Model1_recall)


    #Creating RF model with with 300 estimators
    RF_md2= RandomForestClassifier(max_depth= 3, n_estimators= 300).fit(x_train, y_train)
    #Predictingon the model
    RF_pred2= RF_md2.predict_proba(x_test)[:,1]
    RF_pred2
    #Changling likleyhoods to labels
    RF_label2= np.where(RF_pred2 < .1, 0, 1)
    #Computing recall
    Model2_recall= recall_score(y_test, RF_label2)
    md2_results.append(Model2_recall)


    #Creating RF model with 500 estimators
    RF_md3= RandomForestClassifier(max_depth= 3, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred3= RF_md3.predict_proba(x_test)[:,1]
    RF_pred3
    #Changling likleyhoods to labels
    RF_label3= np.where(RF_pred3 < .1, 0, 1)
    #Computing recall
    Model3_recall= recall_score(y_test, RF_label3)
    md3_results.append(Model3_recall)

In [14]:
#Computing averages
print('Average Recall for model 1:', np.mean(md1_results))
print('Average Recall for model 2:',np.mean(md2_results))
print('Average Recall for model 3:',np.mean(md3_results))

Average Recall for model 1: 0.8784285714285716
Average Recall for model 2: 0.8790000000000001
Average Recall for model 3: 0.8793333333333335


In [16]:
#Trying different number of trees
md1_results= list()
md2_results= list()
md3_results= list()

#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']
    
#depth of 5
    
for i in range (0,1000):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Creating RF model with 100 estimators
    RF_md= RandomForestClassifier(max_depth= 5, n_estimators= 100).fit(x_train, y_train)
    #Predictingon the model
    RF_pred= RF_md.predict_proba(x_test)[:,1]
    RF_pred
    #Changling likleyhoods to labels
    RF_label= np.where(RF_pred < .1, 0, 1)
    #Computing recall
    Model1_recall= recall_score(y_test, RF_label)
    md1_results.append(Model1_recall)


    #Creating RF model with with 300 estimators
    RF_md2= RandomForestClassifier(max_depth= 5, n_estimators= 300).fit(x_train, y_train)
    #Predictingon the model
    RF_pred2= RF_md2.predict_proba(x_test)[:,1]
    RF_pred2
    #Changling likleyhoods to labels
    RF_label2= np.where(RF_pred2 < .1, 0, 1)
    #Computing recall
    Model2_recall= recall_score(y_test, RF_label2)
    md2_results.append(Model2_recall)


    #Creating RF model with 500 estimators
    RF_md3= RandomForestClassifier(max_depth=5, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred3= RF_md3.predict_proba(x_test)[:,1]
    RF_pred3
    #Changling likleyhoods to labels
    RF_label3= np.where(RF_pred3 < .1, 0, 1)
    #Computing recall
    Model3_recall= recall_score(y_test, RF_label3)
    md3_results.append(Model3_recall)

In [17]:
#Computing averages
print('Average Recall for model 1:', np.mean(md1_results))
print('Average Recall for model 2:',np.mean(md2_results))
print('Average Recall for model 3:',np.mean(md3_results))

Average Recall for model 1: 0.9297619047619048
Average Recall for model 2: 0.9307619047619049
Average Recall for model 3: 0.9306666666666668


In [3]:
import xgboost as xgb 
#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']

results= list()
results2= list()
results3= list()

#depth of 3 and LR .1

for i in range(0,1000):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 3, n_estimators= 500, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)
    
    
    #creating model
    xgb_md2= xgb.XGBClassifier(max_depth= 3, n_estimators= 300, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred2= xgb_md2.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label2= np.where(xgb_pred2 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall2= recall_score(y_test, xgb_label2)
    results2.append(xgb_recall2)
    
    
    #creating model
    xgb_md3= xgb.XGBClassifier(max_depth= 3, n_estimators= 100, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred3= xgb_md3.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label3= np.where(xgb_pred3 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall3= recall_score(y_test, xgb_label3)
    results3.append(xgb_recall3)

In [5]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))
print('Average Recall for xgb model:', np.mean(results2))
print('Average Recall for xgb model:', np.mean(results3))

Average Recall for xgb model: 0.859952380952381
Average Recall for xgb model: 0.8882857142857145
Average Recall for xgb model: 0.9440952380952381


In [7]:
import xgboost as xgb 
#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']

results= list()
results2= list()
results3= list()

#depth of 5 and LR .1

for i in range(0,1000):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 5, n_estimators= 500, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)
    
    
    #creating model
    xgb_md2= xgb.XGBClassifier(max_depth= 5, n_estimators= 300, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred2= xgb_md2.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label2= np.where(xgb_pred2 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall2= recall_score(y_test, xgb_label2)
    results2.append(xgb_recall2)
    
    
    #creating model
    xgb_md3= xgb.XGBClassifier(max_depth= 5, n_estimators= 100, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred3= xgb_md3.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label3= np.where(xgb_pred3 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall3= recall_score(y_test, xgb_label3)
    results3.append(xgb_recall3)

In [8]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))
print('Average Recall for xgb model:', np.mean(results2))
print('Average Recall for xgb model:', np.mean(results3))

Average Recall for xgb model: 0.8463809523809526
Average Recall for xgb model: 0.8568571428571429
Average Recall for xgb model: 0.8947619047619049


In [10]:
import xgboost as xgb 
#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']

results= list()
results2= list()
results3= list()

#depth of 7 and LR .1

for i in range(0,1000):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 7, n_estimators= 500, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)
    
    
    #creating model
    xgb_md2= xgb.XGBClassifier(max_depth= 7, n_estimators= 300, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred2= xgb_md2.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label2= np.where(xgb_pred2 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall2= recall_score(y_test, xgb_label2)
    results2.append(xgb_recall2)
    
    
    #creating model
    xgb_md3= xgb.XGBClassifier(max_depth= 7, n_estimators= 100, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred3= xgb_md3.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label3= np.where(xgb_pred3 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall3= recall_score(y_test, xgb_label3)
    results3.append(xgb_recall3)

In [11]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))
print('Average Recall for xgb model:', np.mean(results2))
print('Average Recall for xgb model:', np.mean(results3))

Average Recall for xgb model: 0.8502380952380955
Average Recall for xgb model: 0.8572380952380955
Average Recall for xgb model: 0.8828571428571429


In [12]:
import xgboost as xgb 
#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']

results= list()
results2= list()
results3= list()

#depth 3 and LR 1

for i in range(0,1000):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 3, n_estimators= 500, learning_rate= 1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)
    
    
    #creating model
    xgb_md2= xgb.XGBClassifier(max_depth= 3, n_estimators= 300, learning_rate= 1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred2= xgb_md2.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label2= np.where(xgb_pred2 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall2= recall_score(y_test, xgb_label2)
    results2.append(xgb_recall2)
    
    
    #creating model
    xgb_md3= xgb.XGBClassifier(max_depth= 3, n_estimators= 100, learning_rate= 1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred3= xgb_md3.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label3= np.where(xgb_pred3 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall3= recall_score(y_test, xgb_label3)
    results3.append(xgb_recall3)

In [13]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))
print('Average Recall for xgb model:', np.mean(results2))
print('Average Recall for xgb model:', np.mean(results3))

Average Recall for xgb model: 0.8043333333333335
Average Recall for xgb model: 0.8094761904761906
Average Recall for xgb model: 0.8266666666666668


In [14]:
import xgboost as xgb 
#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']

results= list()
results2= list()
results3= list()

#depth 5 anf LR 1

for i in range(0,1000):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 5, n_estimators= 500, learning_rate= 1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)
    
    
    #creating model
    xgb_md2= xgb.XGBClassifier(max_depth= 5, n_estimators= 300, learning_rate= 1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred2= xgb_md2.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label2= np.where(xgb_pred2 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall2= recall_score(y_test, xgb_label2)
    results2.append(xgb_recall2)
    
    
    #creating model
    xgb_md3= xgb.XGBClassifier(max_depth= 5, n_estimators= 100, learning_rate= 1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred3= xgb_md3.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label3= np.where(xgb_pred3 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall3= recall_score(y_test, xgb_label3)
    results3.append(xgb_recall3)

In [15]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))
print('Average Recall for xgb model:', np.mean(results2))
print('Average Recall for xgb model:', np.mean(results3))

Average Recall for xgb model: 0.8021904761904761
Average Recall for xgb model: 0.8075714285714286
Average Recall for xgb model: 0.8207619047619048


In [16]:
import xgboost as xgb 
#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']

results= list()
results2= list()
results3= list()

#depth 7 and LR 1

for i in range(0,1000):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 7, n_estimators= 500, learning_rate= 1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)
    
    
    #creating model
    xgb_md2= xgb.XGBClassifier(max_depth= 7, n_estimators= 300, learning_rate= 1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred2= xgb_md2.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label2= np.where(xgb_pred2 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall2= recall_score(y_test, xgb_label2)
    results2.append(xgb_recall2)
    
    
    #creating model
    xgb_md3= xgb.XGBClassifier(max_depth= 7, n_estimators= 100, learning_rate= 1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred3= xgb_md3.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label3= np.where(xgb_pred3 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall3= recall_score(y_test, xgb_label3)
    results3.append(xgb_recall3)

In [17]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))
print('Average Recall for xgb model:', np.mean(results2))
print('Average Recall for xgb model:', np.mean(results3))

Average Recall for xgb model: 0.807904761904762
Average Recall for xgb model: 0.8112380952380953
Average Recall for xgb model: 0.8227142857142857


In [7]:
import xgboost as xgb 
#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']

results= list()
results2= list()
results3= list()

#depth 3 and LR .8

for i in range(0,1000):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 3, n_estimators= 500, learning_rate= .8).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)
    
    
    #creating model
    xgb_md2= xgb.XGBClassifier(max_depth= 3, n_estimators= 300, learning_rate= .8).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred2= xgb_md2.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label2= np.where(xgb_pred2 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall2= recall_score(y_test, xgb_label2)
    results2.append(xgb_recall2)
    
    
    #creating model
    xgb_md3= xgb.XGBClassifier(max_depth= 3, n_estimators= 100, learning_rate= .8).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred3= xgb_md3.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label3= np.where(xgb_pred3 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall3= recall_score(y_test, xgb_label3)
    results3.append(xgb_recall3)

In [8]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))
print('Average Recall for xgb model:', np.mean(results2))
print('Average Recall for xgb model:', np.mean(results3))

Average Recall for xgb model: 0.8107142857142858
Average Recall for xgb model: 0.8170952380952382
Average Recall for xgb model: 0.8357142857142857


In [9]:
import xgboost as xgb 
#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']

results= list()
results2= list()
results3= list()

for i in range(0,1000):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 5, n_estimators= 500, learning_rate= .8, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)
    
    
    #creating model
    xgb_md2= xgb.XGBClassifier(max_depth= 5, n_estimators= 300, learning_rate= .8, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred2= xgb_md2.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label2= np.where(xgb_pred2 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall2= recall_score(y_test, xgb_label2)
    results2.append(xgb_recall2)
    
    
    #creating model
    xgb_md3= xgb.XGBClassifier(max_depth= 5, n_estimators= 100, learning_rate= .8, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred3= xgb_md3.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label3= np.where(xgb_pred3 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall3= recall_score(y_test, xgb_label3)
    results3.append(xgb_recall3)

In [10]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))
print('Average Recall for xgb model:', np.mean(results2))
print('Average Recall for xgb model:', np.mean(results3))

Average Recall for xgb model: 0.8154285714285714
Average Recall for xgb model: 0.8206190476190477
Average Recall for xgb model: 0.8341428571428572


In [11]:
import xgboost as xgb 
#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']

results= list()
results2= list()
results3= list()

#depth 7 and LR 1

for i in range(0,1000):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 7, n_estimators= 500, learning_rate= .8, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)
    
    
    #creating model
    xgb_md2= xgb.XGBClassifier(max_depth= 7, n_estimators= 300, learning_rate= .8, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred2= xgb_md2.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label2= np.where(xgb_pred2 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall2= recall_score(y_test, xgb_label2)
    results2.append(xgb_recall2)
    
    
    #creating model
    xgb_md3= xgb.XGBClassifier(max_depth= 7, n_estimators= 100, learning_rate= .8, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred3= xgb_md3.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label3= np.where(xgb_pred3 < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall3= recall_score(y_test, xgb_label3)
    results3.append(xgb_recall3)

In [12]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))
print('Average Recall for xgb model:', np.mean(results2))
print('Average Recall for xgb model:', np.mean(results3))

Average Recall for xgb model: 0.8206666666666668
Average Recall for xgb model: 0.8258571428571428
Average Recall for xgb model: 0.8361904761904763
